# TRAPPIST 1

This is an example of how we made our movie of the TRAPPIST-1 system: https://www.youtube.com/watch?v=WS5UxLHbUKc

You should check out [MakingAMovie.ipynb](MakingAMovie.ipynb) and [WritingMIDIFiles.ipynb](WritingMIDIFiles.ipynb) for an introduction. 

It makes sense to start with our `write_png` function, since the variables/flags that we need for that will have to be worked into how we choreograph our movie. The idea is that it should plot all the particles on a nice background, then make the planet markers bigger close to transit, and shoot out lines when neighboring planets overtake one another, which fade over time. We add a `plotparticles` list for the particles we want plotted (so we can stagger them in), and flags for whether we want to animate the transits and conjunctions:

In [1]:
import PIL # reminder that this is a requirement
from scipy.misc import imread
import matplotlib.pyplot as plt

def write_png(params):
    sim = rebound.Simulation.from_file(params['filename'])
    ps = sim.particles
    plotparticles = params['plotparticles']       # particles that should be displayed
    plottransits = params['plottransits']         # whether we should highlight transits
    plotconjunctions = params['plotconjunctions'] # whether we should draw lines for conjunctions
    color = params['color']                       # colors for each of the particles (including the star)
  
    if sim.t < transits.events[0]['time']:        # only plot particles if the first transit has
        plotparticles = []
        show_orbit = False
    else:
        show_orbit = True
        
    # First make a color list that only includes the particles we want to plot
    coloriterator = [color[i] for i in plotparticles]
    lw=3 # linewidth

    fig = rebound.OrbitPlot(sim, figsize=(8,8), show_orbit=show_orbit, color=coloriterator, lw=lw, plotparticles=plotparticles)
    ax = fig.axes[0] # turn off axes
    ax.axis('off')

    if plottransits:
        refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter
                      # we want markers to match that when far from transit
        for i in plotparticles: # overplot markers so planets have their own color
            ax.scatter(ps[i].x, ps[i].y, s=refsize, color=color[i], marker='o', zorder=4)
            scale=ps[i].a/3     # length scale for making dots bigger
            size=refsize
            if ps[i].x > 0 and np.abs(ps[i].y)/scale < 1: # increase size when particle is within `scale` of y=0, and on right
                size *= 1+6*np.exp(-np.abs(ps[i].y)/scale)
                ax.scatter(ps[i].x, ps[i].y, s=size, color=color[i], marker='o', zorder=5)

    xlim = ax.get_xlim() # Now that all particles are plotted, get the size of the plot
    ylim = ax.get_ylim()
    cscale = 10*xlim[1]  # Take a bigger but relevant length scale for conjunction lines

    if plotconjunctions:    
        fadetimescale = sim.particles[-1].P/3. # timescale for the conjunction lines to dissapear
        # Find all the conjunctions that have already happened and are within fadetimescale of sim.t
        nearby_conjunctions = [conjunction for conjunction in conjunctions.events if sim.t - conjunction['time'] < fadetimescale and sim.t - conjunction['time'] > 0]
        #print(sim.t)
        #print([c['time'] for c in nearby_conjunctions])
        for conjunction in nearby_conjunctions: 
            if conjunction['target'] and conjunction['target'] + 1 in plotparticles: # only plot if both are visible
                # we now have to reload our binary and integrate to the time of conjunction to get the x and y values of the inner planet to plot our line
                filename = "../binaries/trappist.bin"
                sim2 = rebound.Simulation.from_file(filename)
                sim2.t = 0
                ss.rescale_time(sim2, sim2.particles[-1].P)
                sim2.integrate(conjunction['time'])
                x = sim2.particles[conjunction['target']].x
                y = sim2.particles[conjunction['target']].y
                ax.plot([0, cscale*x], [0,cscale*y], lw=5, color=color[conjunction['target']], alpha=max(1.-(sim.t-conjunction['time'])/fadetimescale,0.), zorder=1)

    bkg = imread('../images/US_background_image.png')
    ax.imshow(bkg, zorder=0, extent=xlim+ylim)

    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(params['frame_ctr']), transparent=True, dpi=300)
    plt.close(fig)  

Now we make our movie. We begin by importing what we need, and load our REBOUND binary simulation of the TRAPPIST-1 system that we want to animate/sonify. For convenience, we redefine the time in the simulation to 0, and rescale time so that every orbit of the outermost planet corresponds to one unit of time in the simulation (which we define as one beat in the music).

In [2]:
import sys
sys.path.append('../')
import systemsounds as ss
import numpy as np
import rebound

filename = "../binaries/trappist.bin"
sim = rebound.Simulation.from_file(filename)
sim.t = 0
ss.rescale_time(sim, sim.particles[-1].P)

We chose to begin the movie at 30 beats per minute. We have to convert this to simulation time per second. Above we rescaled time so that one orbit of the outer planet is one unit of simulation time, so 30 bpm / 60 = 0.5 beats per second. We also assign colors to each of the particles in the simulation.

In [3]:
bpm = 30
frames = ss.FrameRecorder(sim, time_per_sec=bpm/60)
transits = ss.EventRecorder(sim, lambda sim, i: sim.particles[i].y)
conjunctions = ss.EventRecorder(sim, lambda sim, i: np.sin(sim.particles[i].theta - sim.particles[i+1].theta))

# The commented out colors are the colors in the video, but we invert the colors in the post-processing, so we use
# the corresponding inverse colors instead
# actualcolors = ['None','red','darkorange','forestgreen','cyan','deepskyblue','violet','darkviolet']
frames.color = [None, 'cyan','dodgerblue','hotpink','crimson','chocolate','forestgreen','yellowgreen']

We also start our MIDI file now, so that we can enter the right tempo information as we go. We will add all the notes at the end:

In [4]:
from midiutil import MIDIFile

midifile = MIDIFile(adjust_origin=True)
midifile.addTempo(track=0, time=sim.t, tempo=bpm) 

We now choreograph the movie. We begin by staggering in the planets from the outside in, one bar (4 beats) at a time. We want to show and play the planets' transits, so we set `frames.plotparticles` and `transits.targets` respectively. We don't yet want to play the conjunctions, so we set `conjunctions.targets` to an empty list. 

In [5]:
frames.plottransits = True
frames.plotconjunctions = False

planets = list(range(sim.N))
conjunctions.targets = []
for i in range(1,sim.N):
    ps = planets[-i:]
    print("t = {0}\t planets = {1}".format(sim.t, ps))
    frames.plotparticles = ps
    transits.targets = ps
    sim.integrate(sim.t+4)

t = 0.0	 planets = [7]
t = 4.0	 planets = [6, 7]
t = 8.0	 planets = [5, 6, 7]
t = 12.0	 planets = [4, 5, 6, 7]
t = 16.0	 planets = [3, 4, 5, 6, 7]
t = 20.0	 planets = [2, 3, 4, 5, 6, 7]
t = 24.0	 planets = [1, 2, 3, 4, 5, 6, 7]


We now want to go back to showing just the outer two planets, and then stagger in the planets again to highlight their conjunctions. So we leave all the transits playing (`transits.targets`), but stop showing them, and now stagger in `conjunctions.targets` and start displaying them. 

Additionally, we want to speed the movie/music up to 100 bpm linearly over the 6 bars in which we play the conjunctions, so we first calculate what the bpm should be at the beginning of each bar:

In [6]:
planets = list(range(sim.N))
frames.plottransits = False
frames.plotconjunctions = True

conjunctionbars = 6
barbpms = np.linspace(30,100,conjunctionbars+1)
print(barbpms)

[  30.           41.66666667   53.33333333   65.           76.66666667
   88.33333333  100.        ]


We now stagger in the planets, and gradually increase the bpms across each bar so it sounds smooth. Each time we change the bpm (`time_per_sec`), we need to add that tempo change to our MIDI file:

In [7]:
bpms = []

N = 10 # number of gradations per bar for bpm adjustments (so it sounds smoother)   
for j, i in enumerate(range(2,sim.N)):
    ps = planets[-i:]
    print("t = {0}\t bpm = {1}\t planets = {2}".format(sim.t, barbpms[j], ps))
    frames.plotparticles = ps
    conjunctions.targets = ps[:-1] # exclude outermost planet, since it doesn't have an exterior neighbor
    
    times = np.linspace(sim.t,sim.t+4,N,endpoint=True)
    bpms = np.linspace(barbpms[j],barbpms[j+1],N,endpoint=True) # array of bpm values at N points within the current bar
    for time, bpm in zip(times, bpms):
        frames.time_per_sec = bpm/60
        midifile.addTempo(track=0, time=sim.t, tempo=bpm) 
        sim.integrate(time)

t = 28.0	 bpm = 30.0	 planets = [6, 7]
t = 32.0	 bpm = 41.666666666666664	 planets = [5, 6, 7]
t = 36.0	 bpm = 53.33333333333333	 planets = [4, 5, 6, 7]
t = 40.0	 bpm = 65.0	 planets = [3, 4, 5, 6, 7]
t = 44.0	 bpm = 76.66666666666666	 planets = [2, 3, 4, 5, 6, 7]
t = 48.0	 bpm = 88.33333333333333	 planets = [1, 2, 3, 4, 5, 6, 7]


We then play for 4 for more bars, keeping everything the same:

In [8]:
for i in range(4):
    print("t = {0}\t planets = {1}".format(sim.t, ps))
    sim.integrate(sim.t+4)

t = 52.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 56.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 60.0	 planets = [1, 2, 3, 4, 5, 6, 7]
t = 64.0	 planets = [1, 2, 3, 4, 5, 6, 7]


Finally, we reduce the tempo back down to 30 bpm, and play for 3 bars (12 beats):

In [9]:
bpm = 30
frames.time_per_sec = bpm/60
sim.integrate(sim.t+12)
print("t = {0}\t planets = {1}".format(sim.t, ps))

t = 80.0	 planets = [1, 2, 3, 4, 5, 6, 7]


# MIDI File

We now write the MIDI file. We scale all transit notes to the outermost planet, which we assign to a C4 note (MIDI note 48). We then manually assign notes to the conjunctions.

In [10]:
transit_notes = ss.calc_midi_notes(sim.particles, ref_note=48, ref_ID=-1)
conjunction_notes = [0, 33, 35, 20, 18, 14, 12]

for transit in transits.events:
    midifile.addNote(track=0, channel=transit['target'], pitch=transit_notes[transit['target']], time=transit['time'], duration=1, volume=100)
for conjunction in conjunctions.events:
        midifile.addNote(track=0, channel=sim.N, pitch=conjunction_notes[conjunction['target']], time=conjunction['time'], duration=1, volume=100)
        
with open("./trappist.mid", "wb") as f:
    midifile.writeFile(f)

For the actual movie, we loaded the MIDI in Logic (music mixing software), and assigned each of these conjunctions to a different drum.

# Write images to temporary folder for movie

Now we write the images. This is parallelized, but making matplotlib plots is slow, so this can take a couple hours! (the movie is 2 mins long).

In [ ]:
%%time
from subprocess import call
call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
res = pool.map(write_png, frames.events)

# Outputting a movie file

You now have all the frames for the movie in `systemsounds/jupyter_examples/tmp/pngs`, as well as the MIDI file `systemsounds/jupyter_examples/trappist.mid`, and can stitch them together into a movie using your favorite software. For some ideas and options, see [MovieEditingSoftware.ipynb](MovieEditingSoftware.ipynb)
